**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import random
import numpy as np
import io
import base64
from IPython.display import HTML
import cv2
import json

!pip install scikit-video
import skvideo.io


import keras
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Dropout, Flatten
from keras import optimizers

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is used to choose what action to do from a given state s.

If the function is called in the testing step, we want to have the best possible action, that is to say the action that maximise the expected reward. That is why it calls "learned_act".

If the function is called in the training step, we need to explore the environement to find the best paths. If we always take the best action, we won't collect more information and we won't learn interesting strategy. If we only use random strategy, we won't necessarly converge toward the best path. That is way, we need to do a compromise between exploration and explotation. epsilon is essential because it controls this tradeoff. We take a random action with a probability epsilon and we take the best action with a probability 1-epsilon.

Finally,  the exploration ratio epsilon affects the learning performances.





***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]  = -1
        self.position[:,0:2]  = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T = 200
temperature = 0.3

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array "position" is used to known where is the rat. It stores 1 in coordinates (x, y) if the rat is in the grid at (x, y).  It stores 0 in coordinates (x, y) if the rat is not in the grid at (x, y).  It stores -1 in coordinates (x, y) if the rat can't access the position (x, y) in the grid (edge condition).

The array "board" contains the rewards. So it is used to known where are the cheese and the poisonous cells (distributed randomly). It stores 0.5 in coordinates (x, y) if a cheese is in the grid at (x, y).  It stores -1 in coordinates (x, y) if poisonous cell is in the grid at (x, y).  It stores 0 in coordinates (x, y) if the rat has already gone to the position (x, y) in the grid.


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
      super(RandomAgent, self).__init__()
      pass

    def learned_act(self, s):
      ####
      action = random.randint(0, self.n_action) #uniform distribution : choose randomly one of the four possible actions
      return(action)
      ####

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        ##### FILL IN HERE
        state = env.reset() #restart and get an initial state
        game_over = False #stop control
        win = 0 #counter for positive rewards
        lose = 0 #counter for negative rewards
        while not game_over:
            action = agent.act(state) #choose an action
            prev_state = state
            state, reward, game_over = env.act(action) #perform the chosen action
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        ####
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [34]:
epochs_test = 11

# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 5.5/11.0. Average score (-5.5)
Win/lose count 8.5/12.0. Average score (-4.5)
Win/lose count 5.5/6.0. Average score (-3.1666666666666665)
Win/lose count 7.0/13.0. Average score (-3.875)
Win/lose count 2.5/10.0. Average score (-4.6)
Win/lose count 6.5/16.0. Average score (-5.416666666666667)
Win/lose count 9.5/10.0. Average score (-4.714285714285714)
Win/lose count 9.0/13.0. Average score (-4.625)
Win/lose count 4.0/8.0. Average score (-4.555555555555555)
Win/lose count 8.5/8.0. Average score (-4.05)
Win/lose count 9.0/19.0. Average score (-4.590909090909091)
Final score: -4.590909090909091


In [0]:
HTML(display_videos('random10.mp4'))

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Let  π  be a policy.

By definition :
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
By shifting index :
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{t=1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \sum_{t=0}^{T-1}\gamma^{t+1}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
We obtain the expected result :
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

---

We assume that the optimal policy $\pi^*$ exists. The optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}
\begin{equation*}
Q^*(s,a)=\max_{s', a'}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a') \> .
\end{equation*}
\begin{equation*}
Q^*(s,a)=\max_{a'}E_{s'\sim \pi^*(.|s,a)}[ r(s,a)+\gamma Q^{\pi}(s',a') \> .
\end{equation*}
\begin{equation*}
Q^*(s,a)=E_{s'\sim \pi^*(.|s,a)}[ r(s,a)+\gamma \max_{a'} Q^{\pi}(s',a') \> .
\end{equation*}



---
We note the deep Q with weights \theta:
\begin{equation*}
Q^(s, a, \theta) = Q^{\pi}(s, a).
\end{equation*}

The objective of the agent is to maximise the maximize cumulative intrinsic reward. So we can define \mathcal{L}(\theta) as the mean-squared error in Q-values.
\begin{equation*}
\mathcal{L}(\theta)=E\Vert \max_{\theta} Q^*(s,a,\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{\theta} \max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}



***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = []

    def remember(self, m):
      ###
      self.memory.append(m)
      return
      ###
      
    def random_access(self):
      ###
      return(random.choice(self.memory))
      ###

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
      ###     
      s = np.array(s).reshape((-1, 5, 5, self.n_state))
      act_values = self.model.predict(s) #prediction of the reward values for the given state
      chosen_action = np.argmax(act_values[0]) #choose the action that maximise the expected reward
      return(chosen_action) 
      ###

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        ######## FILL IN
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        next_states  = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q     = np.zeros((self.batch_size, 4))

        # fill matrices input_states and next_states
        data_list = []
        for i in range(self.batch_size):
            data = self.memory.random_access()
            data_list.append(data)
            input_states[i,:,:,:] = data[0]
            next_states[i,:,:,:]  = data[1]
            
        # predictions
        input_predictions = self.model.predict(input_states)
        target_q = input_predictions.copy()
        next_predictions = self.model.predict(next_states)
        
        #Qlearning to update target_q matrix
        for i in range(self.batch_size):
            data = data_list[i]    
            target_q[i, data[2]] = data[3] + self.discount * np.max(next_predictions[i])
  
        ########
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(x=input_states, y=target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

    ###
    def set_eps(self, eps):
        self.epsilon = eps
        return
    ###
    
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ###
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(24, input_dim=50, activation="relu"))
        model.add(Dense(48, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(4))
        
        model.compile(optimizers.sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        ###
        self.model = model
        

In [13]:
epochs_train = 201 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/201 | Loss 0.0035 | Win/lose count 1.0/4.0 (-3.0)
Epoch 001/201 | Loss 0.0026 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/201 | Loss 0.0084 | Win/lose count 3.0/5.0 (-2.0)
Epoch 003/201 | Loss 0.0007 | Win/lose count 3.5/4.0 (-0.5)
Epoch 004/201 | Loss 0.0078 | Win/lose count 2.0/6.0 (-4.0)
Epoch 005/201 | Loss 0.0036 | Win/lose count 4.5/3.0 (1.5)
Epoch 006/201 | Loss 0.0098 | Win/lose count 6.0/11.0 (-5.0)
Epoch 007/201 | Loss 0.0038 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/201 | Loss 0.0055 | Win/lose count 4.5/1.0 (3.5)
Epoch 009/201 | Loss 0.0133 | Win/lose count 2.0/0 (2.0)
Epoch 010/201 | Loss 0.0063 | Win/lose count 3.0/2.0 (1.0)
Epoch 011/201 | Loss 0.0075 | Win/lose count 1.5/2.0 (-0.5)
Epoch 012/201 | Loss 0.0093 | Win/lose count 4.5/3.0 (1.5)
Epoch 013/201 | Loss 0.0128 | Win/lose count 3.0/4.0 (-1.0)
Epoch 014/201 | Loss 0.0026 | Win/lose count 3.0/6.0 (-3.0)
Epoch 015/201 | Loss 0.0057 | Win/lose count 4.0/4.0 (0.0)
Epoch 016/201 | Loss 0.0036 | Win/lose count 3.0

In [14]:
HTML(display_videos('fc_train200.mp4'))

In [15]:
epochs_test = 101
test(agent, env, epochs_test, prefix='fc_test')
HTML(display_videos('fc_test0.mp4'))

Win/lose count 8.0/0. Average score (8.0)
Win/lose count 15.0/2.0. Average score (10.5)
Win/lose count 10.0/3.0. Average score (9.333333333333334)
Win/lose count 3.5/2.0. Average score (7.375)
Win/lose count 9.5/2.0. Average score (7.4)
Win/lose count 7.0/3.0. Average score (6.833333333333333)
Win/lose count 4.0/2.0. Average score (6.142857142857143)
Win/lose count 2.0/1.0. Average score (5.5)
Win/lose count 12.0/5.0. Average score (5.666666666666667)
Win/lose count 1.0/3.0. Average score (4.9)
Win/lose count 9.0/1.0. Average score (5.181818181818182)
Win/lose count 12.0/2.0. Average score (5.583333333333333)
Win/lose count 15.5/4.0. Average score (6.038461538461538)
Win/lose count 5.0/2.0. Average score (5.821428571428571)
Win/lose count 4.0/1.0. Average score (5.633333333333334)
Win/lose count 9.5/2.0. Average score (5.75)
Win/lose count 1.0/1.0. Average score (5.411764705882353)
Win/lose count 6.0/3.0. Average score (5.277777777777778)
Win/lose count 3.5/2.0. Average score (5.078947

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(24, 3, 3, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(16, 3, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))

        model.compile(optimizers.sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        ###
        
        self.model = model

In [17]:
epochs_train=201

env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=0.1, epsilon=0.1, memory_size=2000, batch_size = 32)
train(agent, env,epochs_train, prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (3, 3), input_shape=(5, 5, 2), activation="relu")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  


Epoch 000/201 | Loss 0.0009 | Win/lose count 1.5/2.0 (-0.5)
Epoch 001/201 | Loss 0.0035 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/201 | Loss 0.0018 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/201 | Loss 0.0027 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/201 | Loss 0.0084 | Win/lose count 4.0/6.0 (-2.0)
Epoch 005/201 | Loss 0.0052 | Win/lose count 3.5/6.0 (-2.5)
Epoch 006/201 | Loss 0.0129 | Win/lose count 3.5/4.0 (-0.5)
Epoch 007/201 | Loss 0.0058 | Win/lose count 3.0/3.0 (0.0)
Epoch 008/201 | Loss 0.0112 | Win/lose count 2.5/3.0 (-0.5)
Epoch 009/201 | Loss 0.0034 | Win/lose count 3.5/4.0 (-0.5)
Epoch 010/201 | Loss 0.0054 | Win/lose count 1.0/4.0 (-3.0)
Epoch 011/201 | Loss 0.0083 | Win/lose count 3.0/4.0 (-1.0)
Epoch 012/201 | Loss 0.0024 | Win/lose count 2.0/1.0 (1.0)
Epoch 013/201 | Loss 0.0100 | Win/lose count 5.0/1.0 (4.0)
Epoch 014/201 | Loss 0.0013 | Win/lose count 1.5/1.0 (0.5)
Epoch 015/201 | Loss 0.0011 | Win/lose count 2.0/3.0 (-1.0)
Epoch 016/201 | Loss 0.0032 | Win/lose count 

In [18]:
HTML(display_videos('cnn_train200.mp4'))

In [19]:
epochs_test = 101
test(agent, env, epochs_test, prefix='cnn_test')
HTML(display_videos('cnn_test0.mp4'))

Win/lose count 19.5/2.0. Average score (17.5)
Win/lose count 10.0/0. Average score (13.75)
Win/lose count 14.0/1.0. Average score (13.5)
Win/lose count 5.0/0. Average score (11.375)
Win/lose count 15.0/4.0. Average score (11.3)
Win/lose count 6.0/3.0. Average score (9.916666666666666)
Win/lose count 9.0/3.0. Average score (9.357142857142858)
Win/lose count 10.5/2.0. Average score (9.25)
Win/lose count 5.5/2.0. Average score (8.61111111111111)
Win/lose count 15.0/0. Average score (9.25)
Win/lose count 7.0/1.0. Average score (8.954545454545455)
Win/lose count 6.0/0. Average score (8.708333333333334)
Win/lose count 9.0/1.0. Average score (8.653846153846153)
Win/lose count 10.0/4.0. Average score (8.464285714285714)
Win/lose count 15.0/3.0. Average score (8.7)
Win/lose count 15.0/3.0. Average score (8.90625)
Win/lose count 4.5/0. Average score (8.647058823529411)
Win/lose count 6.0/2.0. Average score (8.38888888888889)
Win/lose count 6.5/2.0. Average score (8.18421052631579)
Win/lose count

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [50]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent_cnn = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix='cnn_test')

print('')
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix='fc_test')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (3, 3), input_shape=(5, 5, 2), activation="relu")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  


Test of the CNN
Win/lose count 13.5/4.0. Average score (9.5)
Win/lose count 6.5/2.0. Average score (7.0)
Win/lose count 6.0/2.0. Average score (6.0)
Win/lose count 10.5/0. Average score (7.125)
Win/lose count 10.0/1.0. Average score (7.5)
Win/lose count 13.0/2.0. Average score (8.083333333333334)
Win/lose count 8.5/2.0. Average score (7.857142857142857)
Win/lose count 11.5/4.0. Average score (7.8125)
Win/lose count 12.5/6.0. Average score (7.666666666666667)
Win/lose count 12.0/5.0. Average score (7.6)
Win/lose count 6.5/1.0. Average score (7.409090909090909)
Win/lose count 10.5/1.0. Average score (7.583333333333333)
Win/lose count 4.5/4.0. Average score (7.038461538461538)
Win/lose count 6.0/5.0. Average score (6.607142857142857)
Win/lose count 9.0/3.0. Average score (6.566666666666666)
Win/lose count 9.5/4.0. Average score (6.5)
Win/lose count 15.0/3.0. Average score (6.823529411764706)
Win/lose count 14.0/1.0. Average score (7.166666666666667)
Win/lose count 6.0/2.0. Average score (

In [51]:
HTML(display_videos('cnn_test0.mp4'))

In [52]:
HTML(display_videos('fc_test0.mp4'))

The most performant approach is the one using CNN layers. 

Both architectures present a defect : the exploration is limited. The agent tends to repeat the same action instead of exploring more the map. It prevents it to collect all the cheese on the map because the agent tends to stay in the same area.

The parameter temperature is used to construct the environment map : the higher the parameter value, the smaller the number of cell without cheese and poison. It has an impact on the behaviors. The agent tends to explore more the map if the parameter temperature is high. This effect is more visible for the CNN architecture.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [70]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        ###
        #decrease eps
        agent.set_epsilon(agent.epsilon/2.0)
        
        ###
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()

        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        ###
        self.malus_position = np.zeros((grid_size,grid_size))
        ###
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]  = -1
        self.position[:,0:2]  = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1

        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = self.board[self.x, self.y]
        if train:
            reward += self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] = -0.3
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]  = -1
        self.position[:,0:2]  = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y]  = 0
        self.board[self.position<0] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state      
      
"""    
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
"""

'    \n## use those samples of code:\n#In train explore:\nstate, reward, game_over = env.act(action, train=True)\n\n## In Environment exploring:\n# You will have to change n_state to 3 because you will use one more layer!\nreward = 0\nif train:\n    reward = -self.malus_position[self.x, self.y]\nself.malus_position[self.x, self.y] = 0.1\n\nreward = reward + self.board[self.x, self.y]\n# 3 "feature" states instead of 2\nstate = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),\n                                self.board.reshape(self.grid_size, self.grid_size,1),\n                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)\n'

In [71]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=0.1, epsilon=0.8, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (3, 3), input_shape=(5, 5, 3), activation="relu")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu")`
  


Epoch 000/201 | Loss 0.0219 | Win/lose count 7.0/49.4999999999999 (-42.4999999999999)
Epoch 001/201 | Loss 0.0102 | Win/lose count 9.199999999999996/52.69999999999985 (-43.49999999999985)
Epoch 002/201 | Loss 0.0067 | Win/lose count 5.6000000000000005/56.09999999999983 (-50.49999999999983)
Epoch 003/201 | Loss 0.0086 | Win/lose count 4.200000000000002/58.599999999999795 (-54.39999999999979)
Epoch 004/201 | Loss 0.0055 | Win/lose count 3.600000000000001/60.19999999999979 (-56.59999999999979)
Epoch 005/201 | Loss 0.0046 | Win/lose count 4.600000000000001/58.39999999999983 (-53.79999999999983)
Epoch 006/201 | Loss 0.0063 | Win/lose count 2.1/58.6999999999998 (-56.599999999999795)
Epoch 007/201 | Loss 0.0053 | Win/lose count 4.600000000000001/55.79999999999982 (-51.19999999999982)
Epoch 008/201 | Loss 0.0052 | Win/lose count 2.4/59.39999999999979 (-56.999999999999794)
Epoch 009/201 | Loss 0.0030 | Win/lose count 3.400000000000001/63.19999999999981 (-59.79999999999981)
Epoch 010/201 | Loss 

In [72]:
HTML(display_videos('cnn_train_explore200.mp4'))

In [73]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 4.0/0. Average score (4.0)
Win/lose count 5.0/0. Average score (4.5)
Win/lose count 18.0/2.0. Average score (8.333333333333334)
Win/lose count 17.5/5.0. Average score (9.375)
Win/lose count 7.5/3.0. Average score (8.4)
Win/lose count 23.5/16.0. Average score (8.25)
Win/lose count 15.5/5.0. Average score (8.571428571428571)
Win/lose count 13.5/6.0. Average score (8.4375)
Win/lose count 19.5/4.0. Average score (9.222222222222221)
Win/lose count 11.0/2.0. Average score (9.2)
Win/lose count 12.5/2.0. Average score (9.318181818181818)
Win/lose count 18.0/6.0. Average score (9.541666666666666)
Win/lose count 21.5/5.0. Average score (10.076923076923077)
Win/lose count 20.0/0. Average score (10.785714285714286)
Win/lose count 18.0/1.0. Average score (11.2)
Win/lose count 13.5/4.0. Average score (11.09375)
Win/lose count 10.5/4.0. Average score (10.823529411764707)
Win/lose count 2.5/3.0. Average score (10.194444444444445)
Win/lose count 28.0/5.0. Average score (10.86842105263157

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***